In [ ]:
from __future__ import unicode_literals
import os
import re
import sys
sys.path.append("..")
import pickle
import util
import pandas as pd
from entity_link.features import feature_select
from stanfordcorenlp import StanfordCoreNLP
from nltk.tokenize import word_tokenize

In [ ]:
class EntityDetect(object):
    def __init__(self):
        self.nlp = StanfordCoreNLP("http://10.60.1.82", port=9999, lang="en")
        print("Stanford CoreNLP Server connnected ...")
        self.tag_list = ["FW", "NN", "NNP", "NNPS", "NNS"]
        self.tag_NN = ["NN", "NNP", "NNPS", "NNS", "JJ"]
        self.keyword_strict = True      
    
    def getKeyWords(self, question):
        keyword_list = set()
    
        word_tag = self.nlp.pos_tag(question)
        ner_tag = self.nlp.ner(question)
        tag_length = len(word_tag)
        ner_length = len(ner_tag)
        
        print(word_tag)
        print(ner_tag)
        
        ## 从词性标注中添加单个关键词
        for item in word_tag:
            if(item[1] in self.tag_list):
                keyword_list.add(item[0])
                
        ## 从词性标注中添加多个关键词
        for i in range(tag_length):
            if(word_tag[i][1] == "FW"):
                string = ""
                while(i < tag_length and word_tag[i][1] == "FW"):
                    string =  string + word_tag[i][0] + " "
                    i = i + 1
                keyword_list.add(string.rstrip(" "))
            
            if(i < tag_length and word_tag[i][1] in self.tag_NN):
                string = ""
                while(i < tag_length and word_tag[i][1] in self.tag_NN):
                    string =  string + word_tag[i][0] + " "
                    i = i + 1
                keyword_list.add(string.rstrip(" "))
            

        ## 从命名实体识别中添加单个关键词
        for item in ner_tag:
            if(item[1] != "O"):
                keyword_list.add(item[0])
        
        ## 从命名实体识别中添加多个关键词
        for i in range(ner_length):
            if(ner_tag[i][1] != "O"):
                tag = ner_tag[i][1] 
                string = ""
                while(i < ner_length and ner_tag[i][1] == tag):
                    string =  string + ner_tag[i][0] + " "
                    i = i + 1
                keyword_list.add(string.rstrip(" "))
                
        if(self.keyword_strict):
            tmp_list = keyword_list.copy()
            for item in tmp_list:
                inflag = False
                for word in keyword_list:
                    if(item in word and item != word):
                        inflag = True
                        break
                if(inflag):
                    keyword_list.remove(item)
    
        return keyword_list

In [ ]:
nlp = EntityDetect()
question = "which indian city was fazil born in"
nlp.getKeyWords(question)

In [ ]:
count = 0 
wordsBag = []
newfile = open("./WordOfBag.txt","w",encoding="UTF-8")
with open("./questions_all.txt","r", encoding="UTF-8") as file:
    for line in file:
        count = count + 1
        if(count % 100 == 0):
            print("line-%d ..." % count)
        question = line[1:-2]
        keywords = nlp.getKeyWords(question)
        for word in keywords:
            wordsBag.append(word)
print("Finished!")
for item in wordsBag:
    newfile.write(item+"\n")
print("File created!")

In [ ]:
dic = {}
with open("./wordBag.txt","r",encoding="UTF-8") as file:
    for line in file:
        word = line[:-1]
        if(word in dic):
            dic[word] = dic[word] + 1
        else:
            dic[word] = 1
print("Finished!")

In [ ]:
word_list = sorted(dic.items(), key=lambda item:item[1], reverse=True)

In [ ]:
import json
json.dump(dic, open("wordbag.json","w"))

In [ ]:
newfile = open("newquestions.txt","w",encoding="UTF-8")
with open("./questions_all.txt","r",encoding="UTF-8") as file:
    for line in file:
        newfile.write(line[1:-2] + "\n")
print("finished!")
newfile.close()

In [ ]:
import json
newdata = json.load(open("../datas/zyt/keyword_frequence.json","r",encoding="UTF-8"))

In [ ]:
import re
from nltk.tokenize import word_tokenize
def word_count(string1, string2):
    count = 0
    list1 = word_tokenize(string1)
    for word in list1:
        count = count + string2.count(word)
    return count    


def get_top_relation(keywords, relations):
    if(relations == None or keywords == None):
        return None
   
    rel_set = set()
    for rel in relations:
        rel_set.add(rel)
    
    rel_scores = {}
    for rel in rel_set:
        ## 如果是Wikidata：仅对英文名计算相似度
        # word_list = word_tokenize(rel.split("#")[0])
        ## 如果是Freebase：需要对每个单词进行处理，并去除开头的 /
        word_list = re.split('[_|/]', rel[1:])
        rel_len = len(word_list)
        rel_scores[rel] = sorted([word_count(x, rel)/rel_len for x in keywords], reverse=True)[0]
    
    ## 根据分数进行排序，得到元组列表，第一个元组中包含关系名和分数
    relation_rank = sorted(rel_scores.items(), key=lambda item:item[1], reverse=True)
    print("---------- relation rank below ----------")
    print("relations: " + str(relation_rank))
    print("---------- relation rank above ----------")
    return relation_rank[0][0]

In [ ]:
keywords = set(['cyclone', 'hainan'])
relations = ['/base/aareas/schema/administrative_area/administrative_area_type',  '/location/location/people_born_here',  '/base/aareas/schema/administrative_area/administrative_parent',  '/location/location/events',  '/location/cn_province/capital',  '/location/location/containedby',  '/meteorology/cyclone_affected_area/cyclones',  '/location/administrative_division/country',  '/common/topic/notable_types',  '/location/location/contains',  '/base/aareas/schema/administrative_area/administrative_children',  '/base/aareas/schema/administrative_area/capital']
get_top_relation(keywords, relations)

In [ ]:
from wikiparser import getMainImage, infoBox
getMainImage("https://en.wikipedia.org/wiki?curid=2723452")

## 需要进行改进的地方有三处
+ 对一些常用词进行mapping(可以认为是模型学习得到)
```
    kind: type, genre, label
    release: track, album, music, song 
    film: produce, direct, director
    people: person
    capital: city
    pass away: die, death
    nationality: country, where from 
```
+ 对关键词进行词干提取的处理
```python    
    import nltk.stem  
    s = nltk.stem.SnowballStemmer('english')  
    s.stem('imaging')
```
+ 如果无法找到关系，可以通过同义词的方式处理
```python
    ## 同义词中的 synset 三元组是(词.词性.序号)
    from nltk.corpus import wordnet
    syns = wordnet.synsets("word")
    syns = wordnet.synsets("word", pos=wordnet.VERB)
    ## 其中一些 pos 如下：
    ADJ = 'a'  ADJ_SAT = 's'  ADV = 'r'  NOUN = 'n'  VERB = 'v' 
```

+ 训练词向量，通过embedding的相似度计算来得到top relation

In [25]:
from nltk.corpus import wordnet
syns = wordnet.synsets("genre")
a = wordnet.synset('genre.n.04')
b = wordnet.synset('genre.n.01')

a class of art (or artistic endeavor) having a characteristic form or technique
a kind of literary or artistic work


In [ ]:
import nltk.stem  
s = nltk.stem.SnowballStemmer('english')  
s.stem('director')